In [1]:
from sage.algebras.flag_algebras import *

In [1]:
def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    """
    Helper function to check is any of the smalls appears in the large, and if yes,
    returns an injection of the small

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    large - a flag, also must be from a theory with edges relation

    OUTPUT:
    empty list ([]) if all smalls is avoided, otherwise [small, mapping] telling how to
    inject small into large with mapping
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

In [3]:
#This is some hack to create the theory for 3-graphs without C5- and K4-
#up to size 7

TG = ThreeGraphTheory
TG.exclude()
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])
fl5 = TG.generate_flags(5)
gs = check_containment([C5m], fl5)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)
len(fl5), len(fl6)

def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        return []

TGp = CombinatorialTheory("No5Cm", _gen, _identify_hypergraph, edges=3)

print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

9 55 1127


In [22]:
#This is code to create colored theories

def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

TT = CombinatorialTheory("ColoredNo5Cm", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

In [30]:
C = TT(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
Cp = TT(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])
Bp = TT(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

B = TT(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])
M = TT(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])
res = TT.optimize_problem(B + (-1)*M, 6, maximize=True, certificate=True, positives=[Cp - Bp, C - 194/1000])

Ftypes constructed in 26.88s
Block sizes done in 0.09s
Block sizes are [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30, -2840, -100]
Calculating product matrices for 20 ftypes and 2840 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]] is complete: : 20it [00:03,  6.05it/s]    
Table calculation done in 3.31s
Target and constraint calculation done in 0.22s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 3.086018246500899e-11Iter:  1 Ap: 1.50e-01 Pobj: -1.0894060e+01 Ad: 1.57e-01 Dobj:  8.7402339e+01 
Iter:  2 Ap: 3.39e-01 Pobj: -4.0023323e+01 Ad: 4.99e-01 Dobj:  5.3467606e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.2484666e+02 Ad: 3.74e-01 Dobj:  3.2272715e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.3556756e+02 Ad: 8.78e-01 Dobj:  4.5564151e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -1.4195552e+02 Ad: 9.23e-01 Dobj:  3.7044966e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.4767705e+02 Ad: 7

In [31]:
res[0]

3.086018246500899e-11

In [7]:
import itertools
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]

f221 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4], [2, 3, 4]])
f311 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4], [0, 3, 4]])
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=be+list(xx))

targ221 = f221.project()
targ311 = f311.project()
targ222 = f222.project()

edge_lower = TGp(3, edges=[[0, 1, 2]]) - 1/4
edge_upper = 1/4 + 1/1000 - TGp(3, edges=[[0, 1, 2]])
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
alpha = 1-(1/100)

res = optimize_problem(TGp, f222, 7, maximize=False, positives=[edge_lower, edge_upper, degree_difference, degree_difference*(-1)], certificate=True)

Ftypes constructed in 61.51s
Block sizes done in 0.63s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -204]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [00:04,  2.93it/s]
Table calculation done in 4.10s
Target and constraint calculation done in 0.29s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.73e-01 Pobj: -8.3448809e+01 Ad: 6.51e-02 Dobj:  5.3454227e-01 
Iter:  2 Ap: 8.03e-01 Pobj: -4.2124663e+02 Ad: 2.77e-01 Dobj:  2.2698892e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.7083161e+02 Ad: 8.39e-01 Dobj:  5.0819956e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.8089785e+02 Ad: 8.78e-01 Dobj:  1.1975895e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.7886145e+02 Ad: 8.51e-01 Dobj:  6.3523286e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -7.5226615e+02 Ad: 4.81e-01 Dobj:  5.8723223e-02 
Iter:  7 Ap: 2.40e-01 Pobj: -6.3095891e+02 Ad: 5.17e

In [11]:
var("a")
solve(a/4 == (4/81 + (1/4 - 2/9)*(1/27)*a), a)

[a == (24/121)]

In [12]:
n(24/121)

0.198347107438017

In [ ]:
0.1949514499078733
0.19835273254267993

In [27]:
#This is example code to create colored theories

def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateCT(k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    if k==2:
        BT = GraphTheory
    if k==3:
        BT = ThreeGraphTheory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs